In [1]:
import IPython.display as display
from IPython.display import HTML
display.display(HTML("<style>.container { width:50% !important; }</style>"))

%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic
%config Completer.use_jedi = False

import pyphi
from visualize_pyphi import *
from visualize_pyphi import utils
from visualize_pyphi.compute import *
import pickle as pkl
from tqdm.auto import tqdm
import numpy as np
import itertools
import toolz
from pyphi.models.subsystem import FlatCauseEffectStructure as sep
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import pandas as pd
pd.set_option('display.max_rows', None)

'''
Check that the config parameters are correct (if not change pyphi_config.yml to match):
REPERTOIRE_DISTANCE = 'maximal-state-first'
IIT_VERSION = 'IIT_4.0_SMALL_PHI'
PARTITION_TYPE = 'TRI'
'''
pyphi.config.REPERTOIRE_DISTANCE,pyphi.config.IIT_VERSION,pyphi.config.PARTITION_TYPE


C = pyphi.Direction.CAUSE
E = pyphi.Direction.EFFECT

directions = [C,E]


/tmp/ipykernel_9854/819117497.py:20: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [148]:
import pyphi
import toolz
from pyphi.models.subsystem import FlatCauseEffectStructure as sep

directions = [pyphi.Direction.CAUSE, pyphi.Direction.EFFECT]


def get_all_miws(subsystem):
    pset = list(pyphi.utils.powerset(subsystem.node_indices, nonempty=True))
    mechanisms = pset
    purviews = pset

    all_miws = []

    for mechanism in tqdm(mechanisms):
        for direction in [CAUSE, EFFECT]:
            candidate_mices = [
                subsystem.find_mice(direction, mechanism, (purview,))
                for purview in purviews
            ]
            # check if each mice is maximally irreducible within
            candidate_mices_MIW = [
                mice
                for mice in candidate_mices
                if mice.phi > 0
                and not any(
                    [
                        all([unit in mice.purview for unit in mice2.purview])
                        and mice2.phi > mice.phi
                        for mice2 in candidate_mices
                    ]
                )
            ]

            all_miws.extend(candidate_mices_MIW)
    return all_miws


def separate_miws_for_direction(miws):
    return [[m for m in miws if m.direction == d] for d in directions]


def get_empty_purv_slots(subsystem, max_mices_with_winners=None):
    slots = [list(pyphi.utils.powerset(subsystem.node_indices, nonempty=True))] * 2
    if max_mices_with_winners:
        slots = [
            [slot for slot in slots[d] if slot not in [m.purview for m in direction]]
            for d, direction in enumerate(max_mices_with_winners)
        ]
    return slots


def get_mices_over_purview_slots(slots, miws, subsystem):
    directions = [pyphi.Direction.CAUSE, pyphi.Direction.EFFECT]
    return [
        [
            [
                mice
                for mice in miws[d]
                if mice.purview == slot and mice.direction == direction
            ]
            for slot in slots[d]
        ]
        for d, direction in enumerate(directions)
    ]


def get_max_mices_over_purview_slots(mices_over_purview_slots):
    return [
        [max(mices_over_slot) for mices_over_slot in direction if mices_over_slot]
        for direction in mices_over_purview_slots
    ]


def get_conflicts(max_mices_over_purview_slots, subsystem):
    slots = list(pyphi.utils.powerset(subsystem.node_indices, nonempty=True))
    directions = [pyphi.Direction.CAUSE, pyphi.Direction.EFFECT]
    mices_with_same_mechanism = [
        [
            [
                mice
                for mice in max_mices_over_purview_slots[d]
                if mice.mechanism == slot and mice.direction == direction
            ]
            for slot in slots
        ]
        for d, direction in enumerate(directions)
    ]

    conflicts = [
        [mices for mices in direction if len(mices) > 1]
        for direction in mices_with_same_mechanism
    ]
    return conflicts


def get_winners(conflicts):
    winners = []
    for direction in conflicts:
        direction_winners = []
        for conflict in direction:
            purv_length = [len(mice.purview) for mice in conflict]
            if len(set(purv_length)) > 1:
                max_length = max(purv_length)
                winner = [mice for mice in conflict if len(mice.purview) == max_length]
                direction_winners.extend(winner)
            else:
                winner = [mice for mice in conflict if mice.phi == max(conflict).phi]
                #                 if len(winner)>1:
                #                     winner = winner[0]
                direction_winners.extend(winner)
        winners.append(direction_winners)
    return winners


def merge_directional_mice_list(mice_list):
    return sep(list(toolz.concat(mice_list)))


def prune_unlinked_mices(mices):
    mices = merge_directional_mice_list(mices)
    return pyphi.models.FlatCauseEffectStructure(
        [
            mice
            for mice in mices
            if len([m for m in mices if m.mechanism == mice.mechanism]) == 2 and len(set([m.direction for m in mices if m.mechanism==mice.mechanism]))==2
        ]
    )


def get_remaining_miws(miws, max_mices_with_winners=None):
    if max_mices_with_winners:
        used_mechanisms = [
            [m.mechanism for m in direction] for direction in max_mices_with_winners
        ]

        used_purviews = [
            [m.purview for m in direction] for direction in max_mices_with_winners
        ]

        remaining_miws = []
        for d, direction in enumerate(directions):
            remaining_mices_for_direction = []
            for mice in miws[d]:
                if (
                    mice.mechanism not in used_mechanisms[d]
                    and mice.purview not in used_purviews[d]
                ):
                    remaining_mices_for_direction.append(mice)
            remaining_miws.append(remaining_mices_for_direction)

        return remaining_miws
    else:
        return miws


def fill_slots(subsystem):

    slots = get_empty_purv_slots(subsystem)
    miws = separate_miws_for_direction(get_all_miws(subsystem))
    max_mices_with_winners = [[], []]

    while miws[0] and miws[1]:
        slots = get_empty_purv_slots(subsystem, max_mices_with_winners)
        miws = get_remaining_miws(miws, max_mices_with_winners)
        mices_over_purview_slots = get_mices_over_purview_slots(slots, miws, subsystem)
        max_mices_over_purview_slots = get_max_mices_over_purview_slots(
            mices_over_purview_slots
        )
        max_mices_over_purview_slots = [
            max_mices_over_purview_slots[d] + max_mices_with_winners[d]
            for d in range(len(directions))
        ]

        conflicts = get_conflicts(max_mices_over_purview_slots, subsystem)

        winners = get_winners(conflicts)

        max_mices_with_winners = [
            [
                m
                for m in max_mices_over_purview_slots[i]
                if m not in list(toolz.concat(conflicts[i]))
            ]
            + winners[i]
            for i in range(len(winners))
        ]
    else:
        final_ces = prune_unlinked_mices(max_mices_with_winners)
        final_ces.subsystem = subsystem
        return final_ces
    
# ---------------------
# Reflexivity first

def make_bag(all_mices):
    bag = dict()
    for mice in all_mices:
        m = mice.mechanism
        d = mice.direction
        p = mice.purview

        if m in bag.keys():
            if d in bag[m].keys():
                bag[m][d][p] = mice
            else:
                bag[m][d] = dict()
                bag[m][d][p] = mice
        else:
            bag[m] = dict()
            bag[m][d] = dict()
            bag[m][d][p] = mice

    return bag

def get_bag_of_mices(subsystem, mechanisms, purviews, candidate="irreducible"):

    if candidate == "irreducible":
        return {
            mechanism: {
                direction: {
                    purview: subsystem.find_mice(direction, mechanism, (purview,))
                    for purview in purviews
                }
                for direction in [
                    C,
                    E,
                ]
            }
            for mechanism in mechanisms
        }

    elif candidate == "miw":

        # compute candidate mices for every purview
        all_mices = []

        for mechanism in tqdm(mechanisms):
            for direction in [C, E]:
                candidate_mices = [
                    subsystem.find_mice(direction, mechanism, (purview,))
                    for purview in purviews
                ]
                # check if each mice is maximally irreducible within
                candidate_mices_MIW = [
                    mice
                    for mice in candidate_mices
                    if mice.phi > 0
                    and not any(
                        [
                            all([unit in mice.purview for unit in mice2.purview])
                            and mice2.phi > mice.phi
                            for mice2 in candidate_mices
                        ]
                    )
                ]

                all_mices.extend(candidate_mices_MIW)

        bag = make_bag(all_mices)
        return bag
    
def get_mices_from_bag(bag_of_mices,subsystem):
    pset = list(pyphi.utils.powerset(subsystem.node_indices,nonempty=True))
    directions = [pyphi.Direction.CAUSE,pyphi.Direction.EFFECT]
    bag = []
    for m in pset:
        for d in directions:
            for p in pset:
                try:
                    bag.append(bag2[m][d][p])
                except:
                    pass
    return bag


def fill_slots_reflexive(subsystem):
      
    pset = list(pyphi.utils.powerset(subsystem.node_indices,nonempty=True))
    bag_of_mices = get_bag_of_mices(subsystem,pset,pset,candidate="miw")
    flat_bag = []
    for mechanism in tqdm(pset):
        for direction in directions:
            for purview in pset: 
                try:
                    flat_bag.append(bag_of_mices[mechanism][direction][purview])
                except:
                    pass

    reflexive_ds = [
        pyphi.subsystem.Concept(m1.mechanism, m1, m2, subsystem)
        for m1,m2 in itertools.combinations(flat_bag,2)
        if m1.mechanism==m2.mechanism==m1.purview==m2.purview
        and m1.direction == C
        and m2.direction == E
    ]

    reflexive_sep = sep(pyphi.models.CauseEffectStructure(reflexive_ds))

    reflexive_causes = [m.purview for m in reflexive_sep if m.direction==C]
    reflexive_effects = [m.purview for m in reflexive_sep if m.direction==E]
    reflexive_mechanisms = [m.mechanism for m in reflexive_sep]

    remaining_mices = []
    for m in flat_bag:
        if m.direction==C and m.mechanism not in reflexive_mechanisms and m.purview not in reflexive_causes:
            remaining_mices.append(m)
        if m.direction==E and m.mechanism not in reflexive_mechanisms and m.purview not in reflexive_effects:
            remaining_mices.append(m)

    remaining_ces = CES_from_bag_of_mices(make_bag(remaining_mices))

    final_ces = compute.get_linked_ces(list(reflexive_sep)+list(remaining_ces),subsystem)
    final_ces.subsystem=subsystem
    return final_ces

def count_purview_element_states(mices, subsystem):
    #Returns a list containing the number of purview elements in each state 
    #Example output: [[# node1 OFF, # node1 ON], [# node2 OFF, # node2 ON], ...] 
    return [
        [
            len(
                [
                    mice
                    for mice in mices
                    if node in mice.purview
                    and mice.specified_state[0][mice.purview.index(node)] == state
                ]
            )
            for state in [0, 1]
        ]
        for node in subsystem.node_indices
    ]

# Space

In [18]:
net_name = "space_4_lognorm"

th = 1 / 4
exp = 5

s = .8
l = 0.26
f = 0.06
g = 0.0

weights = np.array(
    [
        [s, l, f, g],  # A
        [l, s, l, f],  # B
        [f, l, s, l],  # C
        [g, f, l, s],  # D
        #A, B, C, D,
    ]
)

# weights += np.random.random((6,6))/100
# weights = np.abs(weights)

node_labels = ["A", "B", "C", "D"]
mech_func = [f for f in "ssss"]

network = ng.get_net(
    mech_func,
    weights,
    exp=exp,
    th=th,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=True,
)

state = (0, 0, 0, 0)
subsystem = pyphi.Subsystem(network, state)
utils.pklthis(subsystem,f"example_data/{net_name}_subsystem.pkl")

  0%|          | 0/16 [00:00<?, ?it/s]

Network saved to: space_4_lognorm


### Slot filling MG

In [4]:
final_ces = fill_slots(subsystem)

utils.sepces2df(final_ces,subsystem)

  0%|          | 0/15 [00:00<?, ?it/s]

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.520382
1,A,EFFECT,A,0,0.656129
2,B,CAUSE,AC,00,0.089862
3,B,EFFECT,AC,00,0.128507
4,C,CAUSE,BD,00,0.089862
5,C,EFFECT,BD,00,0.128507
6,D,CAUSE,D,0,0.520382
7,D,EFFECT,D,0,0.656129
8,AB,CAUSE,AB,00,0.071485
9,AB,EFFECT,AB,00,0.069119


In [5]:
# remaining_ces = CES_from_bag_of_mices(make_bag(final_ces))

final_ces = compute.get_linked_ces(final_ces,subsystem)
final_ces.subsystem=subsystem

In [85]:
relations = list(pyphi.relations.relations(subsystem,sep(final_ces),max_degree=3,parallel=True))

  0%|          | 0/1084 [00:00<?, ?it/s]

/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/

/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON


In [86]:
sia = pyphi.big_phi.sia(subsystem,final_ces,relations)

Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

Done submitting tasks


  0%|          | 0/1 [00:00<?, ?it/s]

Evaluating maximal compositional state



KeyboardInterrupt



In [ ]:
existence = np.round(sum(sia.phi_structure.distinctions.phis)+sum([rel.phi for rel in sia.phi_structure.relations]),2)
print(f'Existence={existence}\nIrreducibility={sia.phi}')

In [ ]:
filled_slots = len(
    list(
        toolz.concat(
            [
                [
                    slot
                    for slot in get_empty_purv_slots(subsystem)[d]
                    if slot
                    not in get_empty_purv_slots(
                        subsystem,
                        separate_miws_for_direction(
                            sep(sia.phi_structure.distinctions)
                        ),
                    )[d]
                ]
                for d in range(2)
            ]
        )
    )
)
filled_slots

In [ ]:
purvs_per_order = [len([m for m in sep(sia.phi_structure.distinctions) if len(m.purview)==o]) for o in range(1,len(subsystem)+1)]
purvs_per_order

### Slot filling reflexivity-first

In [25]:
final_ces = fill_slots_reflexive(subsystem)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

In [27]:
relations = list(pyphi.relations.relations(subsystem,sep(final_ces),max_degree=3,parallel=True))

sia = pyphi.big_phi.sia(subsystem,final_ces,relations)

existence = np.round(sum(sia.phi_structure.distinctions.phis)+sum([rel.phi for rel in sia.phi_structure.relations]),2)

print(f'Existence={existence}\nIrreducibility={sia.phi}')

utils.sepces2df(sep(sia.phi_structure.distinctions),subsystem)

  0%|          | 0/1335 [00:00<?, ?it/s]

/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/

/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
2022-02-07 16:46:04,376	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8267


Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

Done submitting tasks


  0%|          | 0/1 [00:00<?, ?it/s]

Evaluating maximal compositional state
Existence=506.76
Irreducibility=0.7895431260903296


,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.520382
1,A,EFFECT,A,0,0.656129
2,B,CAUSE,B,0,0.629285
3,B,EFFECT,B,0,0.713724
4,C,CAUSE,C,0,0.629285
5,C,EFFECT,C,0,0.713724
6,D,CAUSE,D,0,0.520382
7,D,EFFECT,D,0,0.656129
8,AB,CAUSE,AB,00,0.071485
9,AB,EFFECT,AB,00,0.069119


In [14]:
# net_name = "space_4_lognorm"
# with open("example_data/space_4_lognorm_subsystem.pkl", "rb") as f:
#     subsystem = pkl.load(f)

In [15]:
# subsystem_lognorm==subsystem_nearest

In [54]:
# ces = pyphi.compute.ces(subsystem)

In [55]:
mechanisms = list(pyphi.utils.powerset(subsystem.node_indices,nonempty=True))
purviews = list(pyphi.utils.powerset(subsystem.node_indices,nonempty=True))
bag_of_mices = get_bag_of_mices(subsystem,mechanisms,purviews,candidate='miw')

  0%|          | 0/15 [00:00<?, ?it/s]

In [56]:
flat_bag = []
for mechanism in tqdm(mechanisms):
    for direction in [C,E]:
        for purview in mechanisms: 
            try:
                flat_bag.append(bag_of_mices[mechanism][direction][purview])
            except:
                pass

reflexive_ds = [
    pyphi.subsystem.Concept(m1.mechanism, m1, m2, subsystem)
    for m1,m2 in itertools.combinations(flat_bag,2)
    if m1.mechanism==m2.mechanism==m1.purview==m2.purview
    and m1.direction == C
    and m2.direction == E
]

reflexive_sep = sep(pyphi.models.CauseEffectStructure(reflexive_ds))

reflexive_causes = [m.purview for m in reflexive_sep if m.direction==C]
reflexive_effects = [m.purview for m in reflexive_sep if m.direction==E]
reflexive_mechanisms = [m.mechanism for m in reflexive_sep]

remaining_mices = []
for m in flat_bag:
    if m.direction==C and m.mechanism not in reflexive_mechanisms and m.purview not in reflexive_causes:
        remaining_mices.append(m)
    if m.direction==E and m.mechanism not in reflexive_mechanisms and m.purview not in reflexive_effects:
        remaining_mices.append(m)

remaining_ces = CES_from_bag_of_mices(make_bag(remaining_mices))

final_ces = compute.get_linked_ces(list(reflexive_sep)+list(remaining_ces),subsystem)
final_ces.subsystem=subsystem

  0%|          | 0/15 [00:00<?, ?it/s]

In [57]:
relations = list(pyphi.relations.relations(subsystem,sep(final_ces),max_degree=3,parallel=True))

sia = pyphi.big_phi.sia(subsystem,final_ces,relations)

existence = np.round(sum(sia.phi_structure.distinctions.phis)+sum([rel.phi for rel in sia.phi_structure.relations]),2)

print(f'Existence={existence}\nIrreducibility={sia.phi}')

utils.sepces2df(sep(sia.phi_structure.distinctions),subsystem)

  0%|          | 0/1335 [00:00<?, ?it/s]

/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/

/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON


In [61]:
# print('-REFLEXIVITY FIRST-')
# print(f'Network = {net_name}')
# print(f'Existence = {existence}')
# print(f'Irreducibility = {np.round(sia.phi,4)}')
# utils.sepces2df(sep(sia.phi_structure.distinctions),subsystem)

In [77]:
filled_slots = len(
    list(
        toolz.concat(
            [
                [
                    slot
                    for slot in get_empty_purv_slots(subsystem)[d]
                    if slot
                    not in get_empty_purv_slots(
                        subsystem,
                        separate_miws_for_direction(
                            sep(sia.phi_structure.distinctions)
                        ),
                    )[d]
                ]
                for d in range(2)
            ]
        )
    )
)
filled_slots

26

In [81]:
purvs_per_order = [len([m for m in sep(sia.phi_structure.distinctions) if len(m.purview)==o]) for o in range(1,len(subsystem)+1)]
purvs_per_order

[8, 11, 5, 2]

# Ch 3

## 4 nodes

In [152]:
import pyphi
from visualize_pyphi import *
import numpy as np
from pyphi.models.subsystem import FlatCauseEffectStructure as sep

th = 1/4
exp = 5
mu = 1
si = .3
ll = 1
kk = 15
x0 = .7

q = .4
p = .1
s = .8
l = .25
j = 1-q
i = j-.05
b = .01

weights = np.array([
    [s,l,0,i,0,0],#A
    [l,s,l,j,b,0],#B
    [0,l,s,i,0,0],#C
    [b,b,b,q,0,i],#D
    [0,i,0,0,0,0],#I
    [0,0,0,b,0,s],#O
    #A,B,C,D,I,O
  ])

node_labels = ['A','B','C','D','I','O']
mech_func = [f for f in 'sssgll']
network = network_generator.get_net(mech_func, weights, exp=exp, th=th, mu=mu, si=si, l=ll, k=kk, x0=x0, node_labels=node_labels, pickle_network=False)
# utils.pklthis(network,'example_data/ch3_network.pkl')

subsystem = pyphi.Subsystem(network,state=(0,1,0,1,1,1),nodes=(0,1,2,3))
# ces = pyphi.compute.ces(subsystem)

  0%|          | 0/64 [00:00<?, ?it/s]

In [153]:
pset = list(pyphi.utils.powerset(subsystem.node_indices,nonempty=True))

In [164]:
ces = pyphi.compute.ces(subsystem,parallel=True)

In [165]:
count_purview_element_states(sep(ces),subsystem)

[[9, 0], [3, 12], [8, 0], [2, 12]]

In [154]:
# utils.sepces2df(all_miws,subsystem)

In [155]:
bestbig_ces = fill_slots(subsystem)

utils.sepces2df(bestbig_ces,subsystem)

  0%|          | 0/15 [00:00<?, ?it/s]

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.486034
1,A,EFFECT,A,0,0.627340
2,B,CAUSE,B,1,0.232803
3,B,EFFECT,AC,00,0.106082
4,C,CAUSE,C,0,0.486034
5,C,EFFECT,C,0,0.627340
6,AC,CAUSE,D,0,0.003137
7,AC,EFFECT,B,0,0.177767
8,ABC,CAUSE,AC,00,0.070977
9,ABC,EFFECT,ABC,010,0.023489


In [161]:
bag = get_mices_from_bag(get_bag_of_mices(subsystem,pset,pset,candidate="miw"),subsystem)

  0%|          | 0/15 [00:00<?, ?it/s]

In [163]:
count_purview_element_states(bag,subsystem)

[[42, 16], [27, 43], [43, 15], [25, 33]]

In [ ]:
def get_congruence_index(mices,subsystem):
    nodes = subsystem.node_indices
    for node in nodes:
        for state in [0,1]:
            

In [54]:
bestbig_ces.subsystem=subsystem
bestbig_ces = compute.get_linked_ces(bestbig_ces,subsystem)

In [55]:
subsystem.clear_caches()

In [56]:
bestbig_ces_relations = list(pyphi.relations.relations(subsystem,sep(bestbig_ces),parallel=True))

  0%|          | 0/49 [00:00<?, ?it/s]

/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON


In [57]:
subsystem.clear_caches()

In [58]:
bestbig_sia = pyphi.big_phi.sia(subsystem,bestbig_ces,bestbig_ces_relations)

bestbig_existence = np.round(sum(bestbig_sia.phi_structure.distinctions.phis)+sum([rel.phi for rel in bestbig_sia.phi_structure.relations]),2)
print(f'Existence={bestbig_existence}\nIrreducibility={bestbig_sia.phi}')

Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

Done submitting tasks


  0%|          | 0/1 [00:00<?, ?it/s]

Evaluating maximal compositional state
Existence=5.33
Irreducibility=0.0


In [59]:
bestbig_sia.informativeness

0.0

In [60]:
# fig = visualize_ces.plot_ces(
#     subsystem,
#     sep(sia.phi_structure.distinctions),
#     sia.phi_structure.relations,
#     base_rotation=0,
#     base_z_offset=.3,
#     network_name='ch3_bestbigger',
# )
# fig.show()

In [61]:
utils.sepces2df(sep(bestbig_sia.phi_structure.distinctions),subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.492685
1,A,EFFECT,A,0,0.633166
2,B,CAUSE,B,1,0.228257
3,B,EFFECT,AC,00,0.104252
4,C,CAUSE,C,0,0.492685
5,C,EFFECT,C,0,0.633166
6,ABC,CAUSE,AC,00,0.070701
7,ABC,EFFECT,ABC,010,0.023365


In [62]:
len(bestbig_sia.phi_structure.relations)

48

In [63]:
reflexive_ces = fill_slots_reflexive(subsystem)

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [64]:
reflexive_relations = list(pyphi.relations.relations(subsystem,sep(reflexive_ces),parallel=True))

  0%|          | 0/25 [00:00<?, ?it/s]

/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON
/home/mgrasso/pyphi/pyphi/utils.py:86: RuntimeWarning: invalid value encountered in double_scalars
  return abs(x - y) <= constants.EPSILON


In [65]:
subsystem.clear_caches()

In [66]:
reflexive_sia = pyphi.big_phi.sia(subsystem,reflexive_ces,reflexive_relations)

reflexive_existence = np.round(sum(reflexive_sia.phi_structure.distinctions.phis)+sum([rel.phi for rel in reflexive_sia.phi_structure.relations]),2)
print(f'Existence={reflexive_existence}\nIrreducibility={reflexive_sia.phi}')

Done putting relations
Finding maximal compositional state


Submitting tasks: 0it [00:00, ?it/s]

Done submitting tasks


  0%|          | 0/1 [00:00<?, ?it/s]

Evaluating maximal compositional state
Existence=4.45
Irreducibility=0.0


In [67]:
# fig = visualize_ces.plot_ces(
#     subsystem,
#     sep(sia.phi_structure.distinctions),
#     sia.phi_structure.relations,
#     base_rotation=0,
#     base_z_offset=.3,
#     network_name='ch3_reflexivity',    
# )
# fig.show()

In [68]:
utils.sepces2df(sep(reflexive_sia.phi_structure.distinctions),subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.492685
1,A,EFFECT,A,0,0.633166
2,B,CAUSE,B,1,0.228257
3,B,EFFECT,B,1,0.346988
4,C,CAUSE,C,0,0.492685
5,C,EFFECT,C,0,0.633166
6,ABC,CAUSE,AC,00,0.070701
7,ABC,EFFECT,ABC,010,0.023365


In [69]:
len(reflexive_sia.phi_structure.relations)

22

In [70]:
# fig = visualize_ces.plot_ces(
#     subsystem,
#     sep(pruned_sia_ces),
#     pruned_sia_rels,
#     base_rotation=0,
#     base_z_offset=.3,
# )
# fig.show()

## 5 nodes

In [ ]:
subsystem = pyphi.Subsystem(network,state=(0,1,0,1,1,1),nodes=(0,1,2,3,5))
ces = pyphi.compute.ces(subsystem)
utils.sepces2df(sep(ces),subsystem)

In [ ]:
relations = list(pyphi.relations.relations(subsystem,sep(ces),max_degree=2))
sia = pyphi.big_phi.sia(subsystem,ces,relations)
sia.phi

In [ ]:
utils.sepces2df(sep(sia.phi_structure.distinctions),subsystem)